In [ ]:
import requests
from bs4 import BeautifulSoup
import smtplib
import pandas as pd
import numpy as np

import re

In [ ]:
### returns only numbers in a string as int 
def only_numerics(seq):
    seq_type= type(seq)
    return int(seq_type().join(filter(seq_type.isdigit, seq)))

In [ ]:
### returns only numbers in a string but still as a string 
def only_numerics_string(seq):
    seq_type= type(seq)
    return seq_type().join(filter(seq_type.isdigit, seq))

In [ ]:
url1 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page1-9.txt"
page1 = requests.get(url1)
soup1 = BeautifulSoup(page1.content, 'html.parser')

In [ ]:
url2 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page10-19.txt"
page2 = requests.get(url2)
soup2 = BeautifulSoup(page2.content, 'html.parser')

In [ ]:
url3 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page20-29.txt"
page3 = requests.get(url3)
soup3 = BeautifulSoup(page3.content, 'html.parser')

In [ ]:
url4 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page30-39.txt"
page4 = requests.get(url4)
soup4 = BeautifulSoup(page4.content, 'html.parser')

In [ ]:
url5 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page40-49.txt"
page5 = requests.get(url5)
soup5 = BeautifulSoup(page5.content, 'html.parser')

In [ ]:
url6 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page50-59.txt"
page6 = requests.get(url6)
soup6 = BeautifulSoup(page6.content, 'html.parser')

In [ ]:
url7 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page60_69.txt"
page7 = requests.get(url7)
soup7 = BeautifulSoup(page7.content, 'html.parser')

In [ ]:
url8 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page70-79.txt"
page8 = requests.get(url8)
soup8 = BeautifulSoup(page8.content, 'html.parser')

In [ ]:
url9 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page80-89.txt"
page9 = requests.get(url9)
soup9 = BeautifulSoup(page9.content, 'html.parser')

In [ ]:
url10 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page90-99.txt"
page10 = requests.get(url10)
soup10 = BeautifulSoup(page10.content, 'html.parser')

In [ ]:
url11 = "https://raw.githubusercontent.com/tsenior97/Cars_dataset/main/page100-112.txt"
page11 = requests.get(url11)
soup11 = BeautifulSoup(page11.content, 'html.parser')

In [ ]:
def list_extracter(x):
  ##### Blank lists for the data
  Transmission = []
  Seats = []
  Fuel_Type = []
  Fuel_Eco = []
  Insurance_Group = []
  CO2 = []
  # Milage = []
  # Year = []

  reserved= []
  href = []

  make = []
  model =[]
  years = []
  mileage = []
  reges = []
  location =[]

  Depsoit =[]
  APR =[]

  Price = []
  Price_diff = []
  Price_month = []
  Price_new= []
  #####

  car = x.find_all(class_='vc-header') # breaking the html into details about each car 
  num_cars = len(x.find_all(class_='vc-header')) # number of cars in the soup
  specs = x.find_all(class_='vc-img-info-column') # breaking the html into details about each car's specs 
  lists = [[] for i in range(num_cars)] 

  for i in range(num_cars):
    ###### First section is looking at price
    mp_text=specs[i].find_all(class_='main-price')[0].get_text()

    if specs[i].find_all(class_='save-price') == []:
      sp_text = None
      Price_diff.append(sp_text)
    else:
      sp_text=specs[i].find_all(class_='save-price')[0].get_text()
      Price_diff.append(only_numerics(sp_text))

    mop_text=specs[i].find_all(class_='monthly-price')[0].get_text()

    Price.append(only_numerics(mp_text))
    # Price_month.append(only_numerics(mop_text)/100) (Probelm assuming all numbers had two digits)
    Price_month.append(float(re.findall(r"[-+]?\d*\.\d+|\d+", mop_text)[0]))

    desp_apr = only_numerics_string(specs[i].find_all(class_='deposit-apr')[0].get_text())

    APR.append(int(desp_apr[-2:])/10)
    Depsoit.append(int(desp_apr[:-2]))
    
    if Price_diff[i] == None:
      Price_new.append(None)
    else:
      Price_new.append(Price[i] + Price_diff[i])
    ######
    ###### This section is breaking the car's specs into its 7 componats 
    for j in range(6):
      vals = specs[i].find_all(class_='spec-value')[j].get_text()
      lists[i].append(vals)
    
    Transmission.append(lists[i][0])
    Seats.append(lists[i][1])
    Fuel_Type.append(lists[i][2])
    Fuel_Eco.append(only_numerics_string(lists[i][3]))
    Insurance_Group.append(lists[i][4])
    CO2.append(only_numerics(lists[i][5]))
    # Milage.append(lists[i][6])
    # Year.append(lists[i][7])
    ######
    ###### Checking wether it is reserved and getting url to the page
    res = specs[i].find_all(class_='reserved-tag')
    if res == []:
        reserved.append(False)
    else:
        reserved.append(True)

    href.append(specs[i].find_all('a')[0].get('href'))
    ######

    ###### Make and model string broken up 
    make.append(car[i].find_all(class_="vc-title")[0].get_text().split()[0])
    model.append(' '.join(car[i].find_all(class_="vc-title")[0].get_text().split()[1:-1]))
    ######
    
    # reg_num = only_numerics_string(string)[-2:]
    # reg_lets = ''.join(filter(str.isalpha, string))[5:10]
    ##### String for year, milage, location and registration is broken up and appended onto there own lists
    string = car[i].find_all(class_='vc-subtitle')[0].get_text()
    years.append(string[0:4])
    mileage.append(int(only_numerics_string(string)[4:-2]))
    reges.append(string.split()[5])

    #if string.split()[-1:] == ''
    #location.append(string.split()[-1:][0])

    location.append(string.split()[-1:][0])
    if location[i] == 'Donington':
        location[i] = 'Castle Donnington'
    elif location[i] == 'Ashfield':
        location[i] = ('Sutton in Ashfield')
    else:
      pass
      
  ##### meg_list is combinign the lists into one
  meg_list = {'Make': make, 'Model': model, 'Registration':reges, 'Location': location, 'Transmission': Transmission, 'Seats': Seats, 'Fuel_Type': Fuel_Type, 'Fuel_Eco': Fuel_Eco, 'Insurance_Group': Insurance_Group,
            'CO2': CO2, 'Milage': mileage, 'Year': years, 'Price(£)': Price, 'Price_diff': Price_diff, 'Price on New(£)': Price_new, 'Price per month': Price_month, 'Depsoit': Depsoit, 'APR': APR,'Reserved': reserved, 'URL': href}
  
  ##### return the soup as a dataframe 
  return pd.DataFrame(meg_list) 

In [ ]:
soups = [soup1, soup2, soup3, soup4, soup5, soup6, soup7, soup8, soup9, soup10, soup11]
pages = []

In [ ]:
for i in range(len(soups)):
  pages.append(list_extracter(soups[i]))

In [ ]:
big_df = pages[0]
for i in range(len(pages)-1):
  big_df = big_df.merge(pages[i+1], how='outer',on=list(pages[0].columns.values))

In [ ]:
big_df.head()

,Make,Model,Registration,Location,Transmission,Seats,Fuel_Type,Fuel_Eco,Insurance_Group,CO2,Milage,Year,Price(£),Price_diff,Price on New(£),Price per month,Depsoit,APR,Reserved,URL
0,CHEVROLET,SPARK LS,BP63NXZ,Cannock,Manual,5,Petrol,55,4E,119,17824,2014,4499,4721.0,9220.0,94.44,0,9.9,False,https://www.availablecar.com/used-cars/chevrol...
1,Toyota,Aygo 1.0 VVT-i Move,AF64HZA,Sutton in Ashfield,Manual,4,Petrol,65,3E,99,51658,2014,5099,5466.0,10565.0,107.03,0,9.9,False,https://www.availablecar.com/used-cars/toyota-...
2,Skoda,Citigo 1.0 60 SE,YB64HVT,Sutton in Ashfield,Manual,4,Petrol,62,1E,105,48599,2014,5099,3906.0,9005.0,107.03,0,9.9,False,https://www.availablecar.com/used-cars/skoda-c...
3,Fiat,Panda 1.2 Pop,NA64YJC,Castle Donnington,Manual,4,Petrol,54,4U,120,25250,2014,5499,3541.0,9040.0,115.43,0,9.9,False,https://www.availablecar.com/used-cars/fiat-pa...
4,Fiat,500 1.2 Colour Therapy,MT15KZY,Leeds,Manual,4,Petrol,58,7U,113,39838,2015,5999,5166.0,11165.0,125.92,0,9.9,False,https://www.availablecar.com/used-cars/fiat-50...


In [ ]:
big_df['Depsoit'].value_counts()

0    1000
Name: Depsoit, dtype: int64

In [ ]:
big_df = big_df.drop('Depsoit', axis=1)

In [ ]:
big_df.to_csv('car_dataframe.csv')
from google.colab import files
files.download("car_dataframe.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>